<목차>
- Image Segmetation을 위한 Foundation Model 모델 설계를 위한 Task 정의
- 데이터 수집 과정
  - Assisted-manual
  - Semi-automatic
  - Fully-automatic
- 모델 구조
  - Image Encoder
  - Prompt Encoder
  - Mask Decoder
  - 성능
- Everything 기능이란?
- 인상적인 후속 주제들
- Predictor

### Image Segmetation 기능을 하는 Foundation Model 모델 설계를 위한 Task 정의
이미지 Segmentation 모델(SAM)을 만들기 위한 Foundation Model을 설계하기 위해서 한 3가지 질문
1. What task will enable zero-shot generalization?  
2. What is the corresponding model architecture?
3. What data can power this task and model? 

- (처음 본 사물도 인식할 수 있는) zero-shot 일반화를 위한 Task는 어떤 Task인가?  
  --> `어떻하면 애매모호한 상황도 모두 대처할 수 있는 모델을 만들 수 있을까? Promptable Segmentation Task를 정의`.  
      점(point), 박스(box), 마스크(mask), 텍스트(text)를 입력으로 받을 수 있게 설계.  
  --> `모호한 Prompt가 주어졌을 때도 합리적인 mask를 출력해야 한다. 모호하다면 연관된 것을 다 확률로 나타낸다`.
  
- 이에 상응하는 모델 구조는 어떻게 되야 하는가?  
  --> Promt를 당연히 지원해야 한다.
  
- 어떤 데이터가 필요한가?  
  --> Promptable Segmentation Task를 위한 Segmentation Mask는 구하기 어렵다.

### 데이터 수집 과정
1. `Assisted-manual`
   - 공개된 Segmentation Dataset을 이용해 SAM 초기 학습
   - 웹 기반 인터페이스에서 초기 학습된 SAM을 이용해 데이터 생성
   - 새로 취득한 Data로 점진적 모델 학습(6회)
   - 120k 이미지로 부터 4.3M Mask 취득  
2. `Semi-automatic`
   - Mask의 종류를 다양화 하는 것을 목표로 함
   - 1단계에서 학습된 신뢰도 높은 Mask를 작업 화면에 표시
   - Annotator들은 그 외 Object를 작업
   - 새로 취득한 Data로 점진적 모델 학습(5회)
   - 180k 이미지로 부터 5.9M Mask 취득  
3. `Fully automatic`
   - 완전 자동화된 Annotation 단계
   - 이미지에 32 x 32 Regular Grid Point를 각 포인트 마다 대응되는 Mask가 할당된다.
   - 그 중에서 IoU가 높은 Mask만 남긴다.
   - 중복된 Mask 제거 등 후처리 작업 진행
   - 1.1M 이미지로 부터 1.1B Mask 취득

### 모델 구조

![image.png](https://github.com/facebookresearch/segment-anything/raw/main/assets/model_diagram.png?raw=true)
- Image encoder output인 image embedding, Prompt encoder output인 prompt embedding을 `두개의 embedding을 Lightweight Mask decoder에서 결합하여 Mask 예측`
- Prompt encoder + Mask decoder는 50ms 이내로 수행

- Powerful Image Encoder
- Prompt Encoder: Sparse/Dense Prompt로 나뉜다.
  - Sparse Prompt : Points, Boxes, Text를 임베딩 한다.
  - Dense Prompt : Mask을 임베딩 한다.
- Mask Decoder

※ 참고로 오픈소스에서는 Text를 임베딩하는 기능은 없다.

#### Image Encoder

#### Prompt Encoder

In [1]:
# segment-anything\segment_anything\modeling\prompt_encoder.py
class PromptEncoder(nn.Module):

    def __init__(
        self,
        embed_dim: int,
        image_embedding_size: Tuple[int, int],
        input_image_size: Tuple[int, int],
        mask_in_chans: int,
        activation: Type[nn.Module] = nn.GELU,
    ) -> None:
        super().__init__()
    
        self.mask_downscaling = nn.Sequential(
            nn.Conv2d(1, mask_in_chans // 4, kernel_size=2, stride=2),
            LayerNorm2d(mask_in_chans//4),
            activation(),
            nn.Conv2d(mask_in_chans // 4, mask_in_chans, kernel_size=2, stride=2),
            LayerNorm2d(mask_in_chans),
            activation()
            nn.Conv2d(mask_in_chans, embed_dim, kernel_size=1),
        )

    def _get_batch_size(
        self,
        points: Optional[Tuple[torch.Tensor, torch.Tensor]],
        boxes: Optional[torch.Tensor],
        masks: Optional[torch.Tensor],
    ) -> int:
        """Gets the batch size of the output given the batch size of the input
        prompts."""
        if points is not None:
            return points[0].shape[0]
        elif boxes is not None:
            return boxes.shape[0]
        elif masks is not None:
            return masks.shape[0]
        else:
            return 1



    def _embed_masks(self, masks: torch.Tensor) -> torch.Tensor:
        """Embeds mask inputs."""
        mask_embedding = self.mask_downscaling(masks)
        return mask_embedding

    def _get_dive(self) -> torch.device:
        return self.point_embeddings[0].weight.device

    def forward(
        self,
        points: Optional[Tuple[torch.Tensor, torch.Tensor]],
        boxes: Optional[torch.Tensor],
        masks: Optional[torch.Tensor],
    ) -> Tuple[torch.Tensor, torch.Tensor]:
    """Embeds different types of prompts, returning both sparse and dense
        embeddings.

        Arguments:
          points (tuple(torch.Tensor, torch.Tensor) or none): point coordinates
            and labels to embed.
          boxes (torch.Tensor or none): boxes to embed
          masks (torch.Tensor or none): masks to embed

        Returns:
          torch.Tensor: sparse embeddings for the points and boxes, with shape
            BxNx(embed_dim), where N is determined by the number of input points
            and boxes.
            : Points, Boxes의 sparse 임베딩
          torch.Tensor: dense embeddings for the masks, in the shape
            Bx(embed_dim)x(embed_H)x(embed_W)
            : Mask 임베딩
    """
    bs = self._get_batch_size(points, boxes, masks)
    sparse_embeddings = torch.empty((bs, 0, self.embed_dim), device=self._get_device())

    if points is not None:
        coords, labels = points
        point_embeddings = self._embed_points(coords, labels, pad=(boxes is None))
        sparse_embeddings = torch.cat([sparse_embeddings, point_embeddings], dim=1)
    if boxes is not None:
        box_embeddings = self._embed_boxes(boxes)
        sparse_embeddings = torch.cat([sparse_embeddings, box_embeddings], dim=1)
    
    if masks is not None:
        dense_embeddings = self._embed_masks(masks)
    else:
        dense_embeddings = self.no_mask_embed.weight.reshape(1, -1, 1, 1)
            .expand(bs, -1, self.image_embedding_size[0], self.image_embedding_size[1])
    return sparse_embeddings, dense_embeddings


IndentationError: expected an indented block (267457045.py, line 11)

#### Mask Decoder
- Image embedding과 prompt embedding을 받아 마스크를 예측하는 부분.
- Transformer decoder block에 Prompt Self-attention과 Cross-attention을 양방향으로 활용
- ![img.jpg](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbFpHTu%2Fbtr8NcL9qZm%2FcxrpfTQaoVn1ytXq4fAPik%2Fimg.png)

#### 성능
- 256개 A100 GPU로 3 ~ 5일 학습
- 이미지 인코더는 A100에서 0.15s, 632M 파라미터 수.
- 프롬프트 인코더와 마스크 디코더는 CPU에서 0.050s 에 추론 가능하다, 4M 파라미터 수.
- 

Everything 기능이란?
- <참고 강의>: https://www.youtube.com/watch?v=KQ3haqbIaSk&t=2413s
- Everything 기능이란? : 1024개 점(32 x 32)을 64개 씩 16번 나눠서 수행
  - 다시말해 특별한 기능이 아니고, SAM을 16 배치로 나눠서 수행
- Mask와 iou_prediction 추론 및 필터링
  - iou thres 낮은 것 제거
  - stability(?) 낮은 것 제거
  - nms 겹치는 것 제거
- 각 마스크 hole(구멍이 뚤려 있는 마스크는 채워주고)과 island(작게 남아 있는 마스크는 제거)
- 강의자 말에 의하면 아래 코드에 Everything 기능 구현이 있다고 한다.
  - segment-anything\segment_anything\automatic_mask_generator.py

인상적인 후속 주제들
- Inpaint Anything
- CLIP 과의 결합 (자연어를 해석) 
  : (개인 의견) 자연어와 결합해서 
- Segmentation with tracking
  : (개인 의견) 데이터를 수집할때 편할 것 같다.

### Predictor
- segment-anything\segment_anything\predictor.py
- Sam 모델을 사용하기 위한 껍데기라고 보면 된다.

In [ ]:
class SamPredictor:
    def __init__(
        self,
        sam_model: Sam,
    ) -> None:
        """
        Uses SAM to calculate the image embedding for an image, and then
        allow repeated, efficient mask prediction given prompts.

        Arguments:
          sam_model (Sam): The model to use for mask prediction.
        """
        super().__init__()
        self.model = sam_model
        self.transform = ResizeLongestSide(sam_model.image_encoder.img_size)
        self.reset_image()

    def set_image(
        self,
        image: np.ndarray,
        image_format: str = "RGB",
    ) -> None:
        """
        Calculates the image embeddings for the provided image, allowing
        masks to be predicted with the 'predict' method.

        Arguments:
          image (np.ndarray): The image for calculating masks. Expects an
            image in HWC uint8 format, with pixel values in [0, 255].
          image_format (str): The color format of the image, in ['RGB', 'BGR'].
        """
        assert image_format in [
            "RGB",
            "BGR",
        ], f"image_format must be in ['RGB', 'BGR'], is {image_format}."
        if image_format != self.model.image_format:
            image = image[..., ::-1]

        # Transform the image to the form expected by the model
        input_image = self.transform.apply_image(image) # ?
        input_image_torch = torch.as_tensor(input_image, device=self.device)
        input_image_torch = input_image_torch.permute(2, 0, 1).contiguous()[None, :, :, :]

        self.set_torch_image(input_image_torch, image.shape[:2])

    @torch.no_grad() # gpu 안쓴다??
    def set_torch_image(
        self,
        transformed_image: torch.Tensor,
        original_image_size: Tuple[int, ...],
    ) -> None:
        """
        이미지 임베딩을 계산. 'predict' 메소드로 마스크를 예측하게 한다. 

        Arguments:
          transformed_image (torch.Tensor): ResizeLongestSide 클래스에 의해 변환된 1x3xHxW 모양 이미지
          original_image_size (tuple(int, int)): 변환전 이미지 사이즈
        """
        assert (
            len(transformed_image.shape) == 4 
            and transformed_image[1] == 3
            and max(*transformed_image.shape[2:]) == self.model.image_encoder.img_size
        ), f"set_torch_image input must be BCHW with long side {self.model.image_encoder.img_size}."
        self.reset_image()

        self.original_size = original_image_size
        self.input_size = tuple(transform_image.shape[-2:]) # HW
        input_image = self.model.preprocess(transformed_image)
        self.features = self.model.image_encoder(input_image) # 임베딩 된것
        self.is_image_set = True

    def predict(
        self,
        point_coords: Optional[np.ndarray] = None,
        point_labels: Optional[np.ndarray] = None,
        box: Optional[np.ndarray] = None,
        mask_input: Optional[np.ndarray] = None,
        multimask_output: bool = True,
        return_logits: bool = False,
    ) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:    
        """

        Arguments:
          point_coords (np.ndarray or None): N x 2 크기의 배열. N개 점의 (X, Y) 위치. 사용자가 점을 찍는 위치들
          point_labels (np.ndarray or None): point prompts를 위한 N 배열의 라벨 1은 전면, 0은 배경 을 나타낸다.
                                             사물이나 배경이나 지정을 해줘야 되는 구나..


        """


In [ ]:
def select_masks(
    masks: np.ndarray, iou_preds: np.ndarray, num_points: int
) -> Tuple [np.ndarray, np.ndarray]:
    # Determine if we should return the multiclick mask or not from the number of points.
    # The reweighting is used to avoid control flow.
    # Reference: https://github.com/facebookresearch/segment-anything/blob/6fdee8f2727f4506cfbbe553e23b895e27956588/segment_anything/utils/onnx.py#L92-L105
    score_reweight = np.array([1000] + [0] * 2)
    score = iou_preds + (num_points - 2.5) * score_reweight
    best_idx = np.argmax(score)
    masks = np.expand_dims(masks[best_idx, :, :], axis=-1)
    iou_preds = np.expand_dims(iou_preds[best_idx], axis=0)
    return masks, iou_preds
